# Step 2: Viterbi algorithm

In [1]:
import matplotlib.pyplot as plt
from random import randint

We begin by reloading the Data_Munging file

In [2]:
%run 'Data_Munging.ipynb'

<class 'pandas.tslib.Timedelta'>
61.0


IndexError: list index out of range

In [3]:
transitionDistribution

,0,1,4,5,10,13,15,17
0,0.970658,0.001152,0.215548,0.095652,0.001119,0.215909,0.015385,0.461538
1,0.005318,0.998324,0.010601,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.012287,0.000471,0.607774,0.000000,0.002841,0.000000,0.006154,0.000000
5,0.003118,0.000000,0.017668,0.904348,0.000000,0.000000,0.000000,0.000000
10,0.002751,0.000000,0.113074,0.000000,0.995954,0.000000,0.000000,0.000000
13,0.002017,0.000000,0.021201,0.000000,0.000086,0.784091,0.000000,0.025641
15,0.001284,0.000052,0.010601,0.000000,0.000000,0.000000,0.966154,0.000000
17,0.002567,0.000000,0.003534,0.000000,0.000000,0.000000,0.012308,0.512821


In [4]:
observationDistribution

,0,1,4,5,10,13,15,17
1,0.020165,0.001519,0.000000,0.000000,0.000000,0.045455,0.089231,0.025641
5,0.261412,0.003457,0.024735,0.869565,0.001722,0.000000,0.000000,0.000000
6,0.106141,0.000733,0.314488,0.034783,0.000430,0.011364,0.000000,0.000000
7,0.007699,0.000000,0.000000,0.000000,0.000000,0.000000,0.003077,0.102564
8,0.135105,0.000105,0.024735,0.000000,0.000000,0.170455,0.129231,0.589744
9,0.045096,0.000052,0.003534,0.000000,0.000000,0.079545,0.141538,0.025641
12,0.058478,0.982612,0.053004,0.000000,0.000000,0.000000,0.006154,0.025641
13,0.027131,0.003404,0.000000,0.008696,0.000000,0.000000,0.003077,0.025641
14,0.137855,0.003876,0.392226,0.082609,0.001636,0.090909,0.006154,0.051282
17,0.025848,0.000262,0.000000,0.000000,0.000000,0.147727,0.141538,0.076923


In [5]:
initialDistribution = Series([0,0,0,0,1,0,0,0], index = [0,1,4,5,10,13,15,17])
initialDistribution

0     0
1     0
4     0
5     0
10    1
13    0
15    0
17    0
dtype: int64

In [6]:
sensorLabel

1              Microwave
5       Hall-Toilet door
6     Hall-Bathroom door
7          Cups cupboard
8                 Fridge
9        Plates cupboard
12            Front door
13            Dishwasher
14           ToiletFlush
17               Freezer
18         Pans Cupboard
20       Washing machine
23    Groceries Cupboard
24     Hall-Bedroom door
dtype: object

In [7]:
observation = Series([24,24,5,6,14,14,6,9,9,8,8,1,23,1,23,13,9,18,7,9,5,5,12,12,8,23,8,6,14,6,18,18,18,12,18,8,8,7,7,7,8,8,6,6,14,6])
observation

0     24
1     24
2      5
3      6
4     14
5     14
6      6
7      9
8      9
9      8
10     8
11     1
12    23
13     1
14    23
15    13
16     9
17    18
18     7
19     9
20     5
21     5
22    12
23    12
24     8
25    23
26     8
27     6
28    14
29     6
30    18
31    18
32    18
33    12
34    18
35     8
36     8
37     7
38     7
39     7
40     8
41     8
42     6
43     6
44    14
45     6
dtype: int64

In [8]:
observation[5]

14

In [9]:
initialDistribution

0     0
1     0
4     0
5     0
10    1
13    0
15    0
17    0
dtype: int64

In [10]:
activityLabel

0               nothing
1           leave house
4            use toilet
5           take shower
10            go to bed
13    prepare breakfast
15       prepare dinner
17            get drink
dtype: object

In [11]:
def Viterbi(observation, activityLabel, initialDistribution, observationDistribution, transitionDistribution):
    probabilityTable = pd.DataFrame(0.0, index = activityLabel.index, columns = observation.index)
    path = {}
    for activityIndex in activityLabel.index.values:
        probabilityTable[0][activityIndex] = initialDistribution[activityIndex] * observationDistribution[activityIndex][observation[0]]
        path[activityIndex] = [activityIndex]
    
    for observationIndex in range(1,len(observation)):
        newPath = {}
        for activityIndex in activityLabel.index.values:
            (proba, previousActivityInd) = max((probabilityTable[observationIndex-1][previousActivityIndex]*transitionDistribution[previousActivityIndex][activityIndex]*observationDistribution[activityIndex][observation[observationIndex]], previousActivityIndex) for previousActivityIndex in activityLabel.index.values )
            #print proba 
            probabilityTable[observationIndex][activityIndex] = proba
            newPath[activityIndex] = path[previousActivityInd] + [activityIndex]
        path = newPath
    
    (finalProba, finalActivityIndex) = max((probabilityTable[len(observation)-1][activityIndex], activityIndex) for activityIndex in activityLabel.index.values )
    return finalProba, path[finalActivityIndex]
        
        
        

Now we will create the testing set:

In [12]:
daySensor = [sensorData['Start time'][x].day == 27 for x in range(1,len(sensorData))]
daySensor = daySensor + [False]
daySensorData = sensorData[daySensor]
daySensorData.index = np.arange(1,len(daySensorData)+1)
daySensorData


,Start time,End time,ID,Val
1,2008-02-27 05:29:23,2008-02-27 05:30:24,24,1
2,2008-02-27 05:29:30,2008-02-27 05:29:33,6,1
3,2008-02-27 05:30:14,2008-02-27 05:30:15,14,1
4,2008-02-27 05:30:16,2008-02-27 05:30:22,6,1
5,2008-02-27 08:00:17,2008-02-27 08:00:18,24,1
6,2008-02-27 08:00:22,2008-02-28 00:16:40,24,1
7,2008-02-27 08:01:25,2008-02-27 08:01:29,9,1
8,2008-02-27 08:01:31,2008-02-27 08:01:50,23,1
9,2008-02-27 08:01:52,2008-02-27 08:01:53,8,1
10,2008-02-27 08:01:57,2008-02-27 08:02:08,8,1


In [13]:
dayActivity = [activityData['Start time'][x].day == 27 for x in range(1,len(activityData))]
dayActivity = dayActivity + [False]
dayActivityData = activityData[dayActivity]
dayActivityData.index = np.arange(1,len(dayActivityData)+1)
dayActivityData

,Start time,End time,ID
1,2008-02-27 05:29:11,2008-02-27 05:30:26,4
2,2008-02-27 08:01:22,2008-02-27 08:02:58,13
3,2008-02-27 08:13:55,2008-02-27 08:16:07,4
4,2008-02-27 08:16:45,2008-02-27 08:27:20,5
5,2008-02-27 08:34:25,2008-02-27 18:59:03,1
6,2008-02-27 18:59:55,2008-02-27 19:19:08,15
7,2008-02-27 19:19:52,2008-02-27 19:20:11,17
8,2008-02-27 23:03:23,2008-02-27 23:05:40,4


In [14]:
daySensorData['Start time'][1]

Timestamp('2008-02-27 05:29:23')

In [15]:
dayActivityData['Start time'][1]

Timestamp('2008-02-27 05:29:11')

In [16]:
testingFeatureMatrix, testingLabel = convert2LastFiredFeatureMatrix(daySensorData,dayActivityData,60)

In [17]:
testingFeatureMatrix

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,1083.0,1084.0,1085.0,1086.0,1087.0,1088.0,1089.0,1090.0,1091.0,1092.0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,1,1,1,1
17,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
t = testingFeatureMatrix.ix[:,:10]
t

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0
1,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,0
14,0,1,0,0,0,0,0,0,0,0,0
17,0,0,0,0,0,0,0,0,0,0,0


In [31]:
testingObservation = Series(0, index = testingFeatureMatrix.columns)
for i in range(len(testingObservation)): 
    r = np.where(testingFeatureMatrix[i] == 1)
    result = testingFeatureMatrix.index[r[0]]
    testingObservation[i] = result[0]

numOfObservation = len(testingObservation)
obs = testingObservation[:numOfObservation]
validatedPath = testingLabel[:numOfObservation]

In [32]:
proba, path = Viterbi(obs, activityLabel, initialDistribution, observationDistribution, transitionDistribution)


In [33]:
#error rate
errorRate = ( sum(validatedPath != path)/float(len(validatedPath)) ) * 100 
errorRate

26.652298850574713

In [34]:
proba * 100

0.0

In [35]:
errorRate = []
numOfEvents = []
for x in range(1,len(testingObservation),100): 
    obs = testingObservation[:x]
    validatedPath = testingLabel[:x]
    proba, path = Viterbi(obs, activityLabel, initialDistribution, observationDistribution, transitionDistribution)
    errorRate.append(  (sum(validatedPath != path)/float(len(validatedPath))) * 100 )
    numOfEvents.append(x)
    plt.plot(numOfEvents,errorRate)
    plt.xlabel("Number of observations in the training sequence")
    plt.ylabel("Error rate (%)")
    

In [36]:
plt.show()

In [19]:
def crossValidation():
    dayList = [5,27,10,20]
    for d in dayList:
        print d
        daySensor = [sensorData['Start time'][x].day == d for x in range(1,len(sensorData))]
        daySensor = daySensor + [False]
        daySensorData = sensorData[daySensor]
        daySensorData.index = np.arange(1,len(daySensorData)+1)
        print daySensorData.index
        
        dayActivity = [activityData['Start time'][x].day == d for x in range(1,len(activityData))]
        dayActivity = dayActivity + [False]
        dayActivityData = activityData[dayActivity]
        dayActivityData.index = np.arange(1,len(dayActivityData)+1)
        print dayActivityData.index
        
        testingFeatureMatrix, testingLabel = convert2LastFiredFeatureMatrix(daySensorData,dayActivityData,60)
        
        testingObservation = Series(0, index = testingFeatureMatrix.columns)
        for i in range(len(testingObservation)): 
            r = np.where(testingFeatureMatrix[i] == 1)
            result = testingFeatureMatrix.index[r[0]]
            testingObservation[i] = result[0]
            
        errorRate = []
        numOfEvents = []
        for x in range(1,len(testingObservation),250): 
            obs = testingObservation[:x]
            validatedPath = testingLabel[:x]
            proba, path = Viterbi(obs, activityLabel, initialDistribution, observationDistribution, transitionDistribution)
            errorRate.append(  (sum(validatedPath != path)/float(len(validatedPath))) * 100 )
            numOfEvents.append(x)
            plt.plot(numOfEvents,errorRate)
         
    plt.xlabel("Number of observations in the training sequence")
    plt.ylabel("Error rate (%)")
    plt.show()

In [20]:
crossValidation()

27
Int64Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
            18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
            35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51],
           dtype='int64')
Int64Index([1, 2, 3, 4, 5, 6, 7, 8], dtype='int64')
30
Int64Index([], dtype='int64')
Int64Index([], dtype='int64')


IndexError: index out of bounds

In [71]:
activityLabel

0               nothing
1           leave house
4            use toilet
5           take shower
10            go to bed
13    prepare breakfast
15       prepare dinner
17            get drink
dtype: object